In [1]:
import random
import time
import os
import sys

import tensorflow as tf
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm

sys.path.append('../pyfiles')
from Models import AFM, DeepAFM, ADFM

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
        print(e)


1 Physical GPUs, 1 Logical GPUs


In [3]:
df = pd.read_csv('../data/movielens_all.csv')

In [4]:
df

,userId,movieId,tag,label
0,124998.0,76251.0,foul language,1.0
1,131620.0,5707.0,Sally Field,1.0
2,121164.0,30848.0,David E. Durston,0.0
3,91544.0,94466.0,==============,0.0
4,93258.0,924.0,high-tech firms,0.0
...,...,...,...,...
1396687,33119.0,97938.0,AFI 10 (courtroom drama),0.0
1396688,19356.0,593.0,prescient,0.0
1396689,57124.0,67197.0,watch again before hating,0.0
1396690,127138.0,70451.0,add to prospects list,1.0


In [5]:
labels = df['label']
df.drop(columns=['label'], inplace=True)

In [6]:
sparse_columns = ['userId', 'movieId', 'tag']

dense_columns = []

In [7]:
sparse_encoders = {}
for column in sparse_columns:
    sparse_encoders[column] = LabelEncoder()
    df[column] = sparse_encoders[column].fit_transform(df[column].values)

In [8]:
dense_encoders = {}
for column in dense_columns:
    dense_encoders[column] = StandardScaler()
    df[column] = dense_encoders[column].fit_transform(df[column].values.reshape(-1,1))

In [9]:
features = df.nunique().to_dict()

In [10]:
loss_object = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.01)


In [11]:
def loss_function(y_true, y_pred):
    
    loss = loss_object(y_true=y_true, y_pred=y_pred)
    
    rmse = tf.math.sqrt(loss)
    
    return rmse

In [12]:
deepAfm = DeepAFM(features, embedding_size=256, attention_factor=16, rate=0.1, reg=0)


In [13]:
x_train, x_test, y_train, y_test = train_test_split(df, labels, test_size=0.1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)

In [14]:
train_loss = tf.keras.metrics.BinaryCrossentropy(name='train_loss')
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')
train_auc = tf.keras.metrics.AUC(name='train_auc')

val_loss = tf.keras.metrics.BinaryCrossentropy(name='val_loss')
val_accuracy = tf.keras.metrics.BinaryAccuracy(name='val_accuracy')
val_auc = tf.keras.metrics.AUC(name='val_auc')

test_loss = tf.keras.metrics.BinaryCrossentropy(name='test_loss')
test_accuracy = tf.keras.metrics.BinaryAccuracy(name='test_accuracy')
test_auc = tf.keras.metrics.AUC(name='test_auc')



In [15]:
def train_step(sparse_x, dense_x, target):

    with tf.GradientTape() as tape:
        predictions  = deepAfm(sparse_x, dense_x, True)
        loss = loss_function(y_true=target, y_pred=predictions)
                
    gradients = tape.gradient(loss, deepAfm.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, deepAfm.trainable_variables))

    train_loss(y_true=target, y_pred=predictions)
    train_accuracy(y_true=target, y_pred=predictions)
    train_auc(y_true=target, y_pred=predictions)


In [16]:
EPOCHS = 5
BATCH_SIZE = 128
STEPS = x_train.shape[0] // BATCH_SIZE

In [17]:
checkpoint_path = "../models/checkpoints/ml/DeepAFM"

ckpt = tf.train.Checkpoint(transformer=deepAfm,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')

In [18]:
x_train

,userId,movieId,tag
744850,5791,1560,12614
1195630,6833,6122,20272
1234568,6325,319,33907
512663,1730,11616,13429
990990,7388,9412,4983
...,...,...,...
1031135,5943,7103,36814
837234,3935,118,13020
982710,206,11319,15686
34412,7062,13294,21405


In [19]:
for epoch in range(EPOCHS):
    start = time.time()

    train_loss.reset_states()
    train_accuracy.reset_states()
    train_auc.reset_states()

    
    val_loss.reset_states()
    val_accuracy.reset_states()
    val_auc.reset_states()

    for batch in range(STEPS):
        
        sample = x_train.sample(n=BATCH_SIZE)
        indexs = sample.index
        y = y_train[indexs].values.reshape((-1,1))
        sparse_x = {k: np.array(list(v.values())) for k, v in sample[sparse_columns].to_dict().items()}
        dense_x = {k: np.array(list(v.values())) for k, v in sample[dense_columns].to_dict().items()}
        train_step(sparse_x, dense_x, y)

        if batch % 2500 == 0:
            print ('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f} AUC {:.4f}'.format(
              epoch + 1, batch, train_loss.result(), train_accuracy.result(), train_auc.result()))

    for batch in range(x_val.shape[0] // BATCH_SIZE):

        sample = x_val.sample(n=BATCH_SIZE)
        indexs = sample.index
        y = y_val[indexs].values.reshape((-1,1))
        sparse_x = {k: np.array(list(v.values())) for k, v in sample[sparse_columns].to_dict().items()}
        dense_x = {k: np.array(list(v.values())) for k, v in sample[dense_columns].to_dict().items()}
        val_predictions = deepAfm(sparse_x, dense_x, False)

        val_loss(y_true=y, y_pred=val_predictions)
        val_accuracy(y_true=y, y_pred=val_predictions)
        val_auc(y_true=y, y_pred=val_predictions)

    print()
    print('Validation Loss {:.4f} Accuracy {:.4f} AUC {:.4f}'.format(
      val_loss.result(), val_accuracy.result(), val_auc.result()))
    print()

    
    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                             ckpt_save_path))

        print ('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, 
                                                    train_loss.result(), 
                                                    train_accuracy.result()))

        print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))


Epoch 1 Batch 0 Loss 0.6984 Accuracy 0.4375 AUC 0.4183


KeyboardInterrupt: 

In [ ]:
test_loss.reset_states()
test_accuracy.reset_states()
test_auc.reset_states()

for batch in range(x_test.shape[0] // BATCH_SIZE):

        sample = x_test.sample(n=BATCH_SIZE)
        indexs = sample.index
        y = y_test[indexs].values.reshape((-1,1))
        sparse_x = {k: np.array(list(v.values())) for k, v in sample[sparse_columns].to_dict().items()}
        dense_x = {k: np.array(list(v.values())) for k, v in sample[dense_columns].to_dict().items()}
        test_predictions = deepAfm(sparse_x, dense_x, False)

        test_loss(y_true=y, y_pred=test_predictions)
        test_accuracy(y_true=y, y_pred=test_predictions)
        test_auc(y_true=y, y_pred=test_predictions)
        
print('Test Loss {:.4f} Accuracy {:.4f} AUC {:.4f}'.format(
      test_loss.result(), test_accuracy.result(), test_auc.result()))